In [1]:
import tensorflow as tf

/Users/alexeydemyanchuk/cs231nVE/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [33]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [3]:
init = tf.global_variables_initializer()

In [4]:
sess = tf.InteractiveSession()

In [5]:
init.run()

In [6]:
result = f.eval()
print(result)

42


In [10]:
sess.close()

### Normal Equation Linear Regression

In [11]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [12]:
housing = fetch_california_housing()

In [14]:
m, n = housing.data.shape

In [15]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [16]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [17]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [18]:
with tf.Session() as sess:
    theta_val = theta.eval()

In [19]:
theta_val

array([[-3.7465141e+01],
       [ 4.3573415e-01],
       [ 9.3382923e-03],
       [-1.0662201e-01],
       [ 6.4410698e-01],
       [-4.2513184e-06],
       [-3.7732250e-03],
       [-4.2664889e-01],
       [-4.4051403e-01]], dtype=float32)

### Gradient Descent

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
scaler = StandardScaler()

In [22]:
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

#### calculate gradients by hand

In [28]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias,\
                dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1),\
                dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0),\
                    name='theta')
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
                     

Epoch 0 MSE 13.100993
Epoch 100 MSE 4.956968
Epoch 200 MSE 4.8797994
Epoch 300 MSE 4.857674
Epoch 400 MSE 4.8429637
Epoch 500 MSE 4.8323493
Epoch 600 MSE 4.8246217
Epoch 700 MSE 4.8189898
Epoch 800 MSE 4.8148603
Epoch 900 MSE 4.8118467


#### tf autodif 

In [29]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias,\
                dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1),\
                dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0),\
                    name='theta')
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
                     

Epoch 0 MSE 10.67181
Epoch 100 MSE 4.978518
Epoch 200 MSE 4.906018
Epoch 300 MSE 4.877514
Epoch 400 MSE 4.857286
Epoch 500 MSE 4.8426275
Epoch 600 MSE 4.8319826
Epoch 700 MSE 4.824256
Epoch 800 MSE 4.818633
Epoch 900 MSE 4.8145423


#### Using Optimizer

In [31]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias,\
                dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1),\
                dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0),\
                    name='theta')
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=\
                                       learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()c
                     

Epoch 0 MSE 8.257284
Epoch 100 MSE 4.8261814
Epoch 200 MSE 4.8052506
Epoch 300 MSE 4.8035026
Epoch 400 MSE 4.803291
Epoch 500 MSE 4.8032546
Epoch 600 MSE 4.8032546
Epoch 700 MSE 4.803253
Epoch 800 MSE 4.8032546
Epoch 900 MSE 4.8032537


### Placeholders

In [32]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict = {A: [[4,5,6], [7,8,9]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


### Mini Batch GD

In [34]:
reset_graph()

In [35]:
n_epochs = 10
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n +1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42),\
                    name='theta')
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=\
                                              learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [36]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [38]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [39]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [40]:
best_theta

array([[-0.1673944 ],
       [ 0.86515963],
       [ 0.11857292],
       [-0.29859844],
       [ 0.39628983],
       [-0.00649126],
       [ 0.00409026],
       [-0.80076975],
       [-0.7800274 ]], dtype=float32)

### Tensor Board Usage

In [41]:
reset_graph()

In [43]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [44]:
n_epochs = 10
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n +1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42),\
                    name='theta')
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=\
                                              learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [45]:
mse_summary = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [46]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [47]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch,\
                                                          y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    

In [48]:
file_writer.close()

In [49]:
best_theta

array([[-0.1673944 ],
       [ 0.86515963],
       [ 0.11857292],
       [-0.29859844],
       [ 0.39628983],
       [-0.00649126],
       [ 0.00409026],
       [-0.80076975],
       [-0.7800274 ]], dtype=float32)